# DX 704 Week 6 Project

This project will develop a treatment plan for a fictious illness "Twizzleflu".
Twizzleflu is a mild illness caused by a virus.
The main symptoms are a mild fever, fidgeting, and kicking the blankets off the bed or couch.
Mild dehydration has also been reported in more severe cases.
These symptoms typically last 1-2 weeks without treatment.
Word on the internet says that Twizzleflu can be cured faster by drinking copious orange juice, but this has not been supported by evidence so far.
You will be provided with a theoretical model of Twizzleflu modeled as a Markov decision process.
Based on the model, you will compute optimal treatment plans to optimize different criteria, and compare patient discomfort with the different plans.

The full project description, a template notebook, and raw data are available on GitHub: [Project 6 Materials](https://github.com/bu-cds-dx704/dx704-project-06).

We will model Twizzleflu as a Markov decision process.
The model transition probabilities are provided in the file "twizzleflu-transitions.tsv" and the expected rewards are in "twizzleflu-rewards.tsv".
The goal for Twizzleflu is to minimize the expected discomfort of the patient which is expressed as negative rewards in the file.

## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Evaluate a Do Nothing Plan

One of the treatment actions is to do nothing.
Calculate the expected discomfort (not rewards) of a policy that always does nothing.

Hint: for this value calculation and later ones, use value iteration.
The analytical solution has difficulties in practice when there is no discount factor.

In [2]:
# YOUR CHANGES HERE

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import twizzleflu-rewards.tsv and twizzleflu-transitions.tsv
rewards = pd.read_csv('twizzleflu-rewards.tsv', sep='\t')
transitions = pd.read_csv('twizzleflu-transitions.tsv', sep='\t')

In [45]:
rewards.head()

,action,state,reward
0,do-nothing,exposed-1,0.0
1,do-nothing,exposed-2,0.0
2,do-nothing,exposed-3,0.0
3,do-nothing,symptoms-1,-0.5
4,do-nothing,symptoms-2,-1.0


In [47]:
transitions.head()

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.8
1,do-nothing,exposed-1,recovered,0.2
2,do-nothing,exposed-2,exposed-3,0.8
3,do-nothing,exposed-2,recovered,0.2
4,do-nothing,exposed-3,symptoms-1,0.8


In [ ]:
# one step state-action values from a value estimate.
# will use this a lot!

def compute_qT_once(R, P, gamma, v):
    # Rewards + gamma x expected future rewards for each action
        # R means immediate rewards
        # gamma means discount factor
        # P means transition probabilities
        # v means value function
        # R has shape (nA, nS)
        # P has shape (nA, nS, nS)
        # v has shape (nS,)
        # returns Q^T with shape (nA, nS)
    return R + gamma * P @ v


def iterate_values_once(R, P, gamma, v):
    # one step of value iteration
    # returns v_{i+1} given v_i
    # this represents the maximum reward over all actions from computing Q^T
    return np.max(compute_qT_once(R, P, gamma, v), axis=0)

def value_iteration(R, P, gamma, max_iterations=100, tolerance=0.001):
    # initial approximation v_0
        # Vector of all zeros
    v_old = np.zeros(R.shape[-1])

    # Iterate up to max_iterations (default 100)
    for i in range(max_iterations):
        # compute v_{i+1}
            # one step of value iteration
        v_new = iterate_values_once(R, P, gamma, v_old)

        # check if values did not change much (less than tolerance, default 0.001)
        if np.max(np.abs(v_new - v_old)) < tolerance:
            # If there was no significant change, return v_{i+1}
                # Only return if converged (meaning that values did not change much)
            return v_new
        # If there was significant change, continue to next iteration
            # Reassign v_{i+1} to v_i
        v_old = v_new

    # return v_{max_iterations} if not converged
    return v_old

In [142]:
# one step state-action values from a value estimate.
# will use this a lot!

def compute_qT_once_discomfort(R, P, gamma, v, reward_on="state", R_is_cost=False):
    # -R + gamma x expected future rewards for each action
        # -R means immediate discomfort
        # gamma means discount factor
        # P means transition probabilities
        # v means value function
        # R has shape (nA, nS)
        # P has shape (nA, nS, nS)
        # v has shape (nS,)
        # returns Q^T with shape (nA, nS)
    if reward_on == "state":
        C = -R if R_is_cost else (-R)
        Pv = np.einsum('asn,n->as', P, v)
        return C + gamma * Pv
    elif reward_on == "next_state":
        # Build the per-next-state reward matrix
        addend = -R + gamma * v[None, :]
        return np.einsum('asn,an->as', P, addend)
    else:
        raise ValueError("reward_on must be 'state' or 'next_state'")

def iterate_values_once_discomfort(R, P, gamma, v, *, reward_on="next_state", R_is_cost=False):
    # one step of value iteration
    # returns v_{i+1} given v_i
    # this represents the maximum reward over all actions from computing Q^T
    return np.min(compute_qT_once_discomfort(R, P, gamma, v, reward_on=reward_on, R_is_cost=R_is_cost), axis=0)

def value_iteration_discomfort(R, P, gamma, max_iterations=100, tolerance=0.001, reward_on="next_state", R_is_cost=False):
    # initial approximation v_0
        # Vector of all zeros
    v_old = np.zeros(R.shape[-1], dtype=float)

    # Iterate up to max_iterations (default 100)
    for i in range(max_iterations):
        # compute v_{i+1}
            # one step of value iteration
        v_new = iterate_values_once_discomfort(R, P, gamma, v_old, reward_on=reward_on, R_is_cost=R_is_cost)
        # check if values did not change much (less than tolerance, default 0.001)
        if np.max(np.abs(v_new - v_old)) < tolerance:
            # If there was no significant change, return v_{i+1}
                # Only return if converged (meaning that values did not change much)
            return v_new
        # If there was significant change, continue to next iteration
            # Reassign v_{i+1} to v_i
        v_old = v_new

    # return v_{max_iterations} if not converged
    return v_old

In [121]:
a_dn = "do-nothing"

# state index
states = sorted(set(rewards["state"]) | set(transitions["state"]) | set(transitions["next_state"]))
S = {s: i for i, s in enumerate(states)}
ns = len(states)
gamma = 0.9

# Only use do-nothing action
R = np.zeros((1, ns), dtype=float)
P = np.zeros((1, ns, ns), dtype=float)

In [122]:
R

array([[0., 0., 0., 0., 0., 0., 0.]])

In [123]:
P

array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]])

In [124]:
# Fill R for do-nothing (one reward per state; average duplicates if any)
r_dn = (
    rewards
      .groupby("state", as_index=True)["reward"]
      .mean()
)
r_dn

state
exposed-1     0.000
exposed-2     0.000
exposed-3     0.000
recovered     0.000
symptoms-1   -0.625
symptoms-2   -1.250
symptoms-3   -0.625
Name: reward, dtype: float64

In [125]:
# Fill R
for s_name, r in r_dn.items():
    if s_name in S:
        R[0, S[s_name]] = float(r)
R

array([[ 0.   ,  0.   ,  0.   ,  0.   , -0.625, -1.25 , -0.625]])

In [126]:
# Fill P for do-nothing
    # Select only do-nothing transitions
subP = transitions[transitions["action"] == a_dn]
subP

,action,state,next_state,probability
0,do-nothing,exposed-1,exposed-2,0.8
1,do-nothing,exposed-1,recovered,0.2
2,do-nothing,exposed-2,exposed-3,0.8
3,do-nothing,exposed-2,recovered,0.2
4,do-nothing,exposed-3,symptoms-1,0.8
5,do-nothing,exposed-3,recovered,0.2
6,do-nothing,symptoms-1,symptoms-1,0.7
7,do-nothing,symptoms-1,symptoms-2,0.3
8,do-nothing,symptoms-2,symptoms-2,0.7
9,do-nothing,symptoms-2,symptoms-3,0.3


In [127]:
# Sum probabilities for duplicate pairs
subP_agg = (
    subP
      .groupby(["state", "next_state"], as_index=False)["probability"]
      .sum()
)
subP_agg

,state,next_state,probability
0,exposed-1,exposed-2,0.8
1,exposed-1,recovered,0.2
2,exposed-2,exposed-3,0.8
3,exposed-2,recovered,0.2
4,exposed-3,recovered,0.2
5,exposed-3,symptoms-1,0.8
6,recovered,recovered,1.0
7,symptoms-1,symptoms-1,0.7
8,symptoms-1,symptoms-2,0.3
9,symptoms-2,symptoms-2,0.7


In [128]:
# For each state, normalize outgoing probabilities
    # If none, default to self-loop
for s_name in states:
    # Get state index
    s_idx = S[s_name]
    # Get outgoing transition rows
    rows = subP_agg[subP_agg["state"] == s_name]

    # Normalize probabilities
    if len(rows) == 0:
        # No outgoing transitions specified: stay in place
        P[0, s_idx, s_idx] = 1.0
        continue

    # Normalize probabilities by dividing by total
    total = rows["probability"].sum()
    if total <= 0:
        # Degenerate row: also make it a self-loop
        P[0, s_idx, s_idx] = 1.0
        continue

    # Fill in normalized probabilities
    for _, row in rows.iterrows():
        sp_name = row["next_state"]
        if sp_name in S:
            P[0, s_idx, S[sp_name]] = float(row["probability"]) / float(total)

In [129]:
P

array([[[0. , 0.8, 0. , 0.2, 0. , 0. , 0. ],
        [0. , 0. , 0.8, 0.2, 0. , 0. , 0. ],
        [0. , 0. , 0. , 0.2, 0.8, 0. , 0. ],
        [0. , 0. , 0. , 1. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0.7, 0.3, 0. ],
        [0. , 0. , 0. , 0. , 0. , 0.7, 0.3],
        [0. , 0. , 0. , 0.3, 0. , 0. , 0.7]]])

In [144]:
v_optimal_discomfort = value_iteration_discomfort(R, P, gamma, reward_on="next_state", R_is_cost=False)
print("Optimal value function (expected discomfort) for gamma=0.9:")
print(v_optimal_discomfort)
print()

# Pack as dictionary mapping state to value
state_values_discomfort = {states[i]: v for i, v in enumerate(v_optimal_discomfort)}
print("State values (expected discomfort) for gamma=0.9:")
print(state_values_discomfort)

# Convert directly to a dataframe
do_nothing_discomfort = pd.DataFrame(list(state_values_discomfort.items()), columns=['state', 'expected_discomfort'])
do_nothing_discomfort

Optimal value function (expected discomfort) for gamma=0.9:
[2.09407341 2.90929863 4.04150737 0.         4.91952714 3.73423479
 1.18241436]

State values (expected discomfort) for gamma=0.9:
{'exposed-1': np.float64(2.0940734076665395), 'exposed-2': np.float64(2.909298630761604), 'exposed-3': np.float64(4.041507371557486), 'recovered': np.float64(0.0), 'symptoms-1': np.float64(4.919527141459451), 'symptoms-2': np.float64(3.734234792433823), 'symptoms-3': np.float64(1.1824143627916024)}


,state,expected_discomfort
0,exposed-1,2.094073
1,exposed-2,2.909299
2,exposed-3,4.041507
3,recovered,0.000000
4,symptoms-1,4.919527
5,symptoms-2,3.734235
6,symptoms-3,1.182414


Save the expected discomfort by state to a file "do-nothing-discomfort.tsv" with columns state and expected_discomfort.

In [108]:
# YOUR CHANGES HERE

do_nothing_discomfort.to_csv('do-nothing-discomfort.tsv', sep='\t', index=False)

Submit "do-nothing-discomfort.tsv" in Gradescope.

## Part 2: Compute an Optimal Treatment Plan

Compute an optimal treatment plan for Twizzleflu.
It should minimize the expected discomfort (maximize the rewards).

In [ ]:
# YOUR CHANGES HERE

...

Save the optimal actions for each state to a file "minimum-discomfort-actions.tsv" with columns state and action.

In [ ]:
# YOUR CHANGES HERE

...

Submit "minimum-discomfort-actions.tsv" in Gradescope.

## Part 3: Expected Discomfort

Using your previous optimal policy, compute the expected discomfort for each state.

In [ ]:
# YOUR CHANGES HERE

...

Save your results in a file "minimum-discomfort-values.tsv" with columns state and expected_discomfort.

In [ ]:
# YOUR CHANGES HERE

...

Submit "minimum-discomfort-values.tsv" in Gradescope.

## Part 4: Minimizing Twizzleflu Duration

Modifiy the Markov decision process to minimize the days until the Twizzle flu is over.
To do so, change the reward function to always be -1 if the current state corresponds to being sick and 0 if the current state corresponds to being better.
To be clear, the action does not matter for this reward function.


In [ ]:
# YOUR CHANGES HERE

...

Save your new reward function in a file "duration-rewards.tsv" in the same format as "twizzleflu-rewards.tsv".

In [ ]:
# YOUR CHANGES HERE

...

Submit "duration-rewards.tsv" in Gradescope.

## Part 5: Optimize for Shorter Twizzleflu

Compute an optimal policy to minimize the duration of Twizzleflu.

In [ ]:
# YOUR CHANGES HERE

...

Save the optimal actions for each state to a file "minimum-duration-actions.tsv" with columns state and action.

In [ ]:
# YOUR CHANGES HERE

...

Submit "minimum-duration-actions.tsv" in Gradescope.

## Part 6: Shorter Twizzleflu?

Compute the expected number of days sick for each state to a file.

In [ ]:
# YOUR CHANGES HERE

...

Save the expected sick days for each state to a file "minimum-duration-days.tsv" with columns state and expected_sick_days.

In [ ]:
# YOUR CHANGES HERE

...

Submit "minimum-duration-days.tsv" in Gradescope.

## Part 7: Speed vs Pampering

Compute the expected discomfort using the policy to minimize days sick, and compare the results to the expected discomfort when optimizing to minimize discomfort.

In [ ]:
# YOUR CHANGES HERE

...

Save the results to a file "policy-comparison.tsv" with columns state, speed_discomfort, and minimize_discomfort.

In [ ]:
# YOUR CHANGES HERE

...

Submit "policy-comparison.tsv" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.